<a href="https://colab.research.google.com/github/emmex2000/Applied-Statistics-Course-Work/blob/main/GoogleNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import shutil
import urllib
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

In [ ]:
DIR = "drive/MyDrive/Thorax Disease Classification"
os.chdir(DIR)

In [ ]:
df = pd.read_csv("df.csv")
df.head()

,Image Index,Effusion,No Finding,Pneumonia,Pneumothorax
0,00006022_006.png,0,1,0,0
1,00020953_001.png,1,0,0,0
2,00029052_010.png,1,0,0,1
3,00014125_054.png,1,0,0,0
4,00016987_010.png,1,0,0,0


In [ ]:
## split data into train and test
n_train = int(df.shape[0] * .7)
train = df.iloc[:n_train]
test = df.iloc[n_train:]

In [ ]:
col = ['Effusion', 'No Finding', 'Pneumonia', 'Pneumothorax']
col

['Effusion', 'No Finding', 'Pneumonia', 'Pneumothorax']

In [ ]:
import tensorflow as tf
from keras import backend as K 
from keras_preprocessing.image import ImageDataGenerator
import keras

In [ ]:
datagen = ImageDataGenerator(rescale=1./255., validation_split=0.2)



In [ ]:
train_generator = datagen.flow_from_dataframe(dataframe=train,
                                            directory="data/images/",
                                            x_col="Image Index",
                                            y_col=col,
                                            subset="training",
                                            batch_size=128,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="raw",
                                            target_size=(224, 224))



Found 9354 validated image filenames.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 5988 invalid image filename(s) in x_col="Image Index". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [ ]:
valid_generator = datagen.flow_from_dataframe(dataframe=train,
                                            directory="data/images/",
                                            x_col="Image Index",
                                            y_col=col,
                                            subset="validation",
                                            batch_size=128,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="raw",
                                            target_size=(224, 224))



Found 2338 validated image filenames.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 5986 invalid image filename(s) in x_col="Image Index". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255.)
test_generator = test_datagen.flow_from_dataframe(dataframe=test,
                                            directory="data/images/",
                                            x_col="Image Index",
                                            y_col=col,
                                            batch_size=32,
                                            seed=42,
                                            shuffle=False,
                                            class_mode="raw",
                                            target_size=(224, 224))

Found 5016 validated image filenames.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 2562 invalid image filename(s) in x_col="Image Index". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [ ]:
test_label = test_generator.labels
np.save("test_label.npy", test_label)

In [ ]:
# class_names = col

# plt.figure(figsize=(10, 10))
# for images, labels in train_generator.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")

In [ ]:
IMG_SIZE = (224, 224)
IMG_SHAPE = IMG_SIZE + (3,)
IMG_SHAPE

(224, 224, 3)

In [ ]:
base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
# Let's take a look at the base model architecture
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

In [ ]:
image_batch, label_batch = next(iter(train_generator))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(128, 5, 5, 2048)


In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(128, 2048)


In [ ]:
prediction_layer = tf.keras.layers.Dense(4, activation='softmax')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(128, 4)


In [ ]:
base_model.trainable = True

In [ ]:
inputs = tf.keras.Input(shape=(IMG_SHAPE))
x = base_model(inputs, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 220

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

Number of layers in the base model:  311


In [ ]:
model1 = keras.models.clone_model(model)

In [ ]:
#COMPILATION OF MODEL ARCHITECTURE

base_learning_rate = 0.0001
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer = tf.keras.optimizers.Adam(lr=base_learning_rate),
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 4)                 8196      
                                                                 
Total params: 21,810,980
Trainable params: 12,820,868
Non-trainable params: 8,990,112
_________________________________________

In [ ]:
len(model.trainable_variables)

54

In [ ]:
# custom loss

POS_WEIGHT = 10  # multiplier for positive targets, needs to be tuned


def wce(target, output):
    """
    Weighted binary crossentropy between an output tensor 
    and a target tensor. POS_WEIGHT is used as a multiplier 
    for the positive targets.

    Combination of the following functions:
    * keras.losses.binary_crossentropy
    * keras.backend.tensorflow_backend.binary_crossentropy
    * tf.nn.weighted_cross_entropy_with_logits
    """
    # print(f"target: {target}")
    # print(f"output: {output}")
    # transform back to logits
    output = K.clip(output, K.epsilon(), 1 - K.epsilon())
    output = tf.math.log(output / (1 - output))

    # output = tf.constant(output)
    # target = tf.constant(target)
    labels = tf.cast(tf.reshape(target , [-1, 4]), dtype=tf.float32)

    # compute weighted loss
    loss = tf.nn.weighted_cross_entropy_with_logits(labels=labels,
                                                    logits=output,
                                                    pos_weight=POS_WEIGHT)
   # loss = tf.where(tf.is_nan(loss), tf.zeros_like(loss), loss)
    return K.mean(loss)

# def wce(target, output):
   
#     # transform back to logits
#     output = K.clip(output, K.epsilon(), 1 - K.epsilon())
#     output = tf.math.log(output / (1 - output))

#     # output = tf.constant(output)
#     # target = tf.constant(target)
#     labels = tf.cast(tf.reshape(target , [-1, 4]), dtype=tf.float32)

#     # compute weighted loss
#     loss = tf.reduced_mean(tf.nn.weighted_cross_entropy_with_logits(labels=labels, logits=output, pos_weight=POS_WEIGHT))
#     return loss

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
# #AFTER COMPILING THE MODEL, THE MODEL IS TRAINED BELOW
callbacks = [
    # tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True,),
    tf.keras.callbacks.TensorBoard(log_dir='./logs/GoogleNetlog' , histogram_freq=1),
    # tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=3, verbose=1, min_delta=0.0001, min_lr=0, ),
    
]
#AFTER COMPILING THE MODEL, THE MODEL IS TRAINED BELOW


STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size

model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,  validation_steps=STEP_SIZE_VALID, 
                    epochs=100, verbose=1, callbacks=callbacks)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/100
73/73 [==============================] - 225s 3s/step - loss: 0.4535 - accuracy: 0.5983 - val_loss: 0.4144 - val_accuracy: 0.6445
Epoch 2/100
73/73 [==============================] - 253s 3s/step - loss: 0.3982 - accuracy: 0.6607 - val_loss: 0.4028 - val_accuracy: 0.6602
Epoch 3/100
73/73 [==============================] - 250s 3s/step - loss: 0.3590 - accuracy: 0.6994 - val_loss: 0.3924 - val_accuracy: 0.6758
Epoch 4/100
73/73 [==============================] - 208s 3s/step - loss: 0.3007 - accuracy: 0.7590 - val_loss: 0.4373 - val_accuracy: 0.6372
Epoch 5/100
73/73 [==============================] - 207s 3s/step - loss: 0.2356 - accuracy: 0.8079 - val_loss: 0.4286 - val_accuracy: 0.6701
Epoch 6/100
73/73 [==============================] - 207s 3s/step - loss: 0.1566 - accuracy: 0.8795 - val_loss: 0.5943 - val_accuracy: 0.6766
Epoch 7/100
73/73 [==============================] - 209s 3s/step - loss: 0.1019 - accuracy: 0.9241 - val_loss: 0.6109 - val_accuracy: 0.6654
Epoch 

In [ ]:
model.save("models/GoogleNet-model.h5")

In [ ]:
test_label = test_generator.labels

In [ ]:
new_model = keras.models.load_model("models/GoogleNet-model.h5", compile=False)

In [ ]:
prob = new_model.predict_generator(test_generator)
prob

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """Entry point for launching an IPython kernel.


array([[1.0000000e+00, 4.6011751e-34, 1.4752118e-30, 1.0055566e-33],
       [2.7017584e-03, 1.2122653e-02, 9.8321491e-01, 1.9606359e-03],
       [7.7074550e-20, 1.8491099e-22, 4.7596230e-24, 1.0000000e+00],
       ...,
       [2.9955108e-22, 1.0000000e+00, 6.3030504e-23, 6.8255134e-19],
       [1.0000000e+00, 6.3159801e-29, 3.3071547e-20, 1.5174747e-26],
       [1.0000000e+00, 3.8357822e-27, 1.2970562e-23, 3.5248186e-33]],
      dtype=float32)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:

roc_auc_score(test_label, prob)

0.7498533880062219

In [ ]:
roc_auc_score(test_label[:, 0], prob[:, 0])

0.798897227924032

In [ ]:
roc_auc_score(test_label[:, 1], prob[:, 1])

0.8197661476780794

In [ ]:
roc_auc_score(test_label[:, 2], prob[:, 2])

0.5985546056855378

In [ ]:
roc_auc_score(test_label[:, 3], prob[:, 3])

0.7821955707372384

In [ ]:
base_learning_rate = 0.0001
model1.compile(loss=wce,
               optimizer = tf.keras.optimizers.Adam(lr=base_learning_rate),
               metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
#AFTER COMPILING THE MODEL, THE MODEL IS TRAINED BELOW
callbacks = [
    # tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True,),
    tf.keras.callbacks.TensorBoard(log_dir='./logs/GoogleNetlog1' , histogram_freq=1),
    # tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=3, verbose=1, min_delta=0.0001, min_lr=1e-5, ),
    
]
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size

model1.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,  validation_steps=STEP_SIZE_VALID,
                    epochs=100, verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/100
73/73 [==============================] - 209s 3s/step - loss: 3.3659 - accuracy: 0.5143 - val_loss: 3.2696 - val_accuracy: 0.5252
Epoch 2/100
73/73 [==============================] - 197s 3s/step - loss: 3.2596 - accuracy: 0.5189 - val_loss: 3.2643 - val_accuracy: 0.5247
Epoch 3/100
73/73 [==============================] - 195s 3s/step - loss: 3.2580 - accuracy: 0.5183 - val_loss: 3.2690 - val_accuracy: 0.5234
Epoch 4/100
73/73 [==============================] - 193s 3s/step - loss: 3.2554 - accuracy: 0.5197 - val_loss: 3.2787 - val_accuracy: 0.5213
Epoch 5/100
73/73 [==============================] - 193s 3s/step - loss: 3.2572 - accuracy: 0.5185 - val_loss: 3.2800 - val_accuracy: 0.5221
Epoch 6/100
73/73 [==============================] - 194s 3s/step - loss: 3.2550 - accuracy: 0.5189 - val_loss: 3.2742 - val_accuracy: 0.5230
Epoch 7/100
73/73 [==============================] - 196s 3s/step - loss: 3.2541 - accuracy: 0.5166 - val_loss: 3.2862 - val_accuracy: 0.5221
Epoch 

In [ ]:
model1.save("models/GoogleNet-model1.h5")

In [ ]:
new_model1 = keras.models.load_model("models/GoogleNet-model1.h5", compile=False)
prob1 = model1.predict_generator(test_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  


In [ ]:
roc_auc_score(test_label[:, 0], prob1[:, 0])

In [ ]:
roc_auc_score(test_label[:, 1], prob1[:, 1])

In [ ]:
roc_auc_score(test_label[:, 2], prob1[:, 2])

0.5791766652984329

In [ ]:
roc_auc_score(test_label[:, 3], prob1[:, 3])

0.6995390799073791